# DICOM-to-BIDS conversion using `heudiconv` and `singularity`

* This notebook creates jobs for each participant and executes them by looping across participants in this notebook or by creating slurm jobs that can be run in parallel on the slurm cluser
 

#### HISTORY

* 9/9/21 dcosme - initial code

## Do the DICOM to BIDS conversion for new data using existing config setup

* Now we have set up a mapping and edited the `heuristics.py` file we can do dicom2bids conversion with the following Singularity command.

* __N.B.__ Make sure the source folder in:
   ```
   /fmriDataRaw/fmri_data_raw/bbprime
   ```

* The only thing that needs to be changed is the subject identifier:
   ```
   -s BPP00
   ```
   
   it should match the folder name of the subject's data in
   ```
   /fmriDataRaw/fmri_data_raw/bbprime
   ```

## Loop through participants
The code below creates a job that can be executed in this notebook

### Define variables

In [6]:
import os

project = 'bbprime'
job_dir = os.path.join('/data00/projects/' + project + '/scripts/BIDS/jobs')
subs = ['BPP15', 'BPP22', 'BPP26', 'BPP34', 'BPP37', 'BPP30', 'BPP38',
        'BPP41', 'BPP40', 'BPP29', 'BPP43', 'BPP33', 'BPP36', 'BPP42',
        'BPP25', 'BPP44']

job_file_template = r'''#!/bin/bash
singularity run --cleanenv \
    -B /data00/projects/bbprime:/base  \
    -B /fmriDataRaw/fmri_data_raw:/raw \
    /data00/tools/singularity_images/heudiconv_0.8.0 \
    -d /raw/bbprime/{subject}/*/*.dcm \
    -o /base/data/bids_data/ \
    -f heudiconv/code/heuristic.py -s {ID} -c dcm2niix -b --overwrite
'''

### Loop through specified participants

In [ ]:
# make the job directory if it doens't exist
if os.path.exists(job_dir) == False:
    os.mkdir(job_dir)

# loop over participants
for s in subs:
    print('-------------- Creating: {} -------------'.format(s))
    
    file_path = os.path.join(job_dir, 'heudiconv_{}.job').format(s)
    print(file_path)

    with open(file_path.format(s), 'w') as job:
        job.write(job_file_template.format(ID=s, subject='{subject}'))
    
    !bash $file_path


-------------- Creating: BPP15 -------------
/data00/projects/bbprime/scripts/BIDS/jobs/heudiconv_BPP15.job
INFO: Running heudiconv version 0.8.0 latest 0.9.0
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'subject': 'BPP15', 'outdir': '/base/data/bids_data/', 'session': None}
INFO: Processing 3940 dicoms
INFO: Analyzing 3940 dicoms


## Submit jobs using slurm
The code below creates a job that can be run using slurm

### Define variables

In [117]:
project = 'bbprime'
slurm_dir = os.path.join('/data00/slurm_jobs/slurm_' + project)
subs = ['BPP07']

job_file_template = r'''#!/bin/bash
#SBATCH --job-name=heudiconv_{ID}.job
#SBATCH --output=out/heudiconv_{ID}.out
#SBATCH --error=out/heudiconv_{ID}.err
#SBATCH --time=02:00

srun singularity run --cleanenv \
    -B /data00/projects/bbprime:/base  \
    -B /fmriDataRaw/fmri_data_raw:/raw \
    /data00/tools/singularity_images/heudiconv_0.8.0 \
    -d /raw/bbprime/{ID}/*/*.dcm \
    -o /base/data/bids_data/ \
    -f heudiconv/code/heuristic.py -s {ID} -c dcm2niix -b --overwrite
'''

### Loop through specified participants

In [118]:
for s in subs:
    print('-------------- Creating: {} -------------'.format(s))
    
    file_path = os.path.join(slurm_dir, 'heudiconv_{}.job').format(s)
    print(file_path)

    with open(file_path.format(s), 'w') as job:
        job.write(job_file_template.format(ID=s, subject='{subject}'))


-------------- Creating: BPP01 -------------
/data00/slurm_jobs/slurm_bbprime/heudiconv_BPP01.job


### Schedule the jobs on slurm

Login to the slurm cluster:

```
ssh <JANUS_UN>@asc.upenn.edu@cls000
```

This will give you a terminal on the SLURM master node where you can look at the process queue and schedule jobs by pasting the output from the next chunk

In [108]:
print(f"cd {slurm_dir}")
for s in subs:
    print(f"sbatch -D {slurm_dir} -c 8 heudiconv_{s}.job")

cd /data00/slurm_jobs/slurm_bbprime
sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 heudiconv_BPP01.job


In [114]:
!ls /data00/slurm_jobs/slurm_bbprime/

fmriprep_sub-BPP00.job	heudiconv_BPP01.job  out
